<a href="https://colab.research.google.com/github/ninja-marduk/ml_precipitation_prediction/blob/main/models/base_models_STHyMOUNTAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📘 Entrenamiento de Modelos Baseline para Predicción Espaciotemporal de Precipitación Mensual STHyMOUNTAIN

Este notebook implementa modelos baseline para la predicción de precipitaciones usando datos espaciotemporales.

## 🔍 Implementación de Modelos Avanzados y Técnicas de Validación

Además de los modelos tabulares baseline, implementaremos:

1. **Optimización avanzada con Optuna** para los modelos tabulares XGBoost y LightGBM
2. **Validación robusta** mediante:
   - Hold-Out Validation (ya implementada)
   - Cross-Validation (k=5)
   - Bootstrapping (100 muestras)
3. **Modelos de Deep Learning** para capturar patrones espaciales y temporales:
   - Redes CNN para patrones espaciales
   - Redes ConvLSTM para patrones espaciotemporales

El objetivo es proporcionar una evaluación completa de diferentes enfoques de modelado para la predicción de precipitación en regiones montañosas.

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
# Configuración del entorno (compatible con Colab y local)
import os
import sys
from pathlib import Path

# Detectar si estamos en Google Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # Si estamos en Colab, clonar el repositorio
    !git clone https://github.com/username/ml_precipitation_prediction.git
    %cd ml_precipitation_prediction
    # Instalar dependencias necesarias
    !pip install -r requirements.txt
    !pip install xarray netCDF4 optuna matplotlib seaborn lightgbm xgboost scikit-learn
    BASE_PATH = '/content/drive/MyDrive/ml_precipitation_prediction'
else:
    # Si estamos en local, usar la ruta actual
    if '/models' in os.getcwd():
        BASE_PATH = Path('..')
    else:
        BASE_PATH = Path('.')

print(f"Entorno configurado. Usando ruta base: {BASE_PATH}")

# Si BASE_PATH viene como string, lo convertimos
BASE_PATH = Path(BASE_PATH)

# Ahora puedes concatenar correctamente
model_output_dir = BASE_PATH / 'models' / 'output'
model_output_dir.mkdir(parents=True, exist_ok=True)

print(f"Directorio para salida de modelos creado: {model_output_dir}")


Cloning into 'ml_precipitation_prediction'...
fatal: could not read Username for 'https://github.com': No such device or address
[Errno 2] No such file or directory: 'ml_precipitation_prediction'
/content
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
Entorno configurado. Usando ruta base: /content/drive/MyDrive/ml_precipitation_prediction
Directorio para salida de modelos creado: /content/drive/MyDrive/ml_precipitation_prediction/models/output


In [ ]:
# Optimización adaptativa de memoria RAM para Optuna (compatible con Colab y Local)
import gc
import psutil
import os
import tempfile

def get_available_memory():
    """Detecta la memoria RAM disponible en el sistema actual (Colab o local)"""
    try:
        # Obtener memoria disponible usando psutil
        mem_info = psutil.virtual_memory()
        available_mb = mem_info.available / (1024 * 1024)  # Convertir a MB
        total_mb = mem_info.total / (1024 * 1024)  # Convertir a MB
        
        # Detectar si estamos en Colab
        in_colab = 'google.colab' in sys.modules
        if in_colab:
            # En Colab, limitar el uso a un porcentaje conservador
            print("Entorno detectado: Google Colab")
            max_usage_percent = 70  # Usar máximo 70% de la memoria disponible en Colab
        else:
            # En local, también ser conservador pero un poco menos
            print("Entorno detectado: Local")
            max_usage_percent = 80  # Usar máximo 80% de la memoria disponible en local
        
        # Calcular memoria máxima a usar
        max_memory_mb = available_mb * (max_usage_percent / 100)
        
        print(f"Memoria total del sistema: {total_mb:.0f} MB")
        print(f"Memoria disponible: {available_mb:.0f} MB")
        print(f"Memoria máxima a utilizar: {max_memory_mb:.0f} MB ({max_usage_percent}% de la disponible)")
        
        return max_memory_mb
    except Exception as e:
        print(f"Error al obtener información de memoria: {e}")
        # Valor conservador por defecto
        return 2000  # 2GB por defecto si no se puede detectar

def calculate_optimal_batch_size(mem_per_sample_mb, max_memory_mb, min_batch=10):
    """Calcula el tamaño óptimo de lote basado en la memoria disponible"""
    # Estimar cuánta memoria usa cada muestra (con margen de seguridad)
    optimal_batch = int(max_memory_mb / (mem_per_sample_mb * 1.5))
    return max(optimal_batch, min_batch)

def configure_optuna_for_memory_efficiency(max_memory_mb, model_name):
    """Configura parámetros para Optuna basados en la memoria disponible"""
    # Estimar número de trials y paralelismo basado en la memoria
    if max_memory_mb < 1000:  # Menos de 1GB disponible
        n_trials = 15  # Reducir número de trials
        n_jobs = 1     # Sin paralelismo
        use_sqlite = True  # Usar SQLite para ahorrar memoria
    elif max_memory_mb < 4000:  # Menos de 4GB
        n_trials = 25  # Número moderado de trials
        n_jobs = min(2, os.cpu_count() or 2)  # Paralelismo limitado
        use_sqlite = True  # Usar SQLite
    else:  # 4GB o más
        n_trials = 30  # Número normal de trials
        n_jobs = min(3, os.cpu_count() or 2)  # Mejor paralelismo
        use_sqlite = True  # Mantener SQLite para estabilidad
    
    # Definir configuración específica por modelo
    model_config = {
        'RandomForest': {
            'n_estimators_max': min(500, int(50 + max_memory_mb / 100)),  # Adaptar a memoria
            'max_depth_max': min(50, int(10 + max_memory_mb / 200))
        },
        'XGBoost': {
            'tree_method': 'hist',  # Método eficiente en memoria
            'max_depth_max': min(12, 3 + int(max_memory_mb / 1000))
            'grow_policy': 'lossguide'  # Más eficiente en memoria
        },
        'LightGBM': {
            'max_bin': min(255, 63 + int(max_memory_mb / 100)),  # Adaptar precisión
            'num_leaves_max': min(150, 31 + int(max_memory_mb / 100))
        }
    }
    
    # Crear almacenamiento SQLite si es necesario
    storage = None
    if use_sqlite:
        # Crear directorio para almacenamiento Optuna si no existe
        os.makedirs(model_output_dir / 'optuna_storage', exist_ok=True)
        storage = f"sqlite:///{model_output_dir}/optuna_storage/optuna_{model_name}_{int(time.time())}.db"
    
    # Devolver toda la configuración optimizada
    return {
        'n_trials': n_trials,
        'n_jobs': n_jobs,
        'storage': storage,
        'model_config': model_config.get(model_name, {})
    }

def memory_efficient_objective_factory(model_name, X_train, y_train, max_memory_mb, model_config):
    """Crea una función objetivo para Optuna optimizada para memoria"""
    # Configurar espacio de búsqueda adaptado a la memoria disponible
    def objective(trial):
        # Limpiar memoria antes de cada trial
        gc.collect()
        
        # Configuración de parámetros específicos para cada modelo, adaptados a la memoria
        if model_name == 'RandomForest':
            n_estimators_max = model_config.get('n_estimators_max', 500)
            max_depth_max = model_config.get('max_depth_max', 50)
            
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, n_estimators_max),
                'max_depth': trial.suggest_int('max_depth', 5, max_depth_max),
                'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
                'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
                'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
                'random_state': 42,
                # Parámetros para optimizar memoria
                'verbose': 0,
                'n_jobs': 1 if max_memory_mb < 2000 else 2  # Limitar paralelismo según memoria
            }
            model = RandomForestRegressor(**params)
        
        elif model_name == 'XGBoost':
            max_depth_max = model_config.get('max_depth_max', 12)
            tree_method = model_config.get('tree_method', 'hist')
            grow_policy = model_config.get('grow_policy', 'lossguide')
            
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
                'max_depth': trial.suggest_int('max_depth', 3, max_depth_max),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                'subsample': trial.suggest_float('subsample', 0.5, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
                'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
                'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),
                'random_state': 42,
                # Optimización de memoria
                'tree_method': tree_method,
                'grow_policy': grow_policy,
                'verbosity': 0,
                'nthread': 1 if max_memory_mb < 2000 else 2  # Adaptar según memoria
            }
            model = XGBRegressor(**params)
        
        elif model_name == 'LightGBM':
            num_leaves_max = model_config.get('num_leaves_max', 150)
            max_bin = model_config.get('max_bin', 63)
            
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                'num_leaves': trial.suggest_int('num_leaves', 20, num_leaves_max),
                'max_depth': trial.suggest_int('max_depth', -1, 15),
                'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 5, 100),
                'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
                'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
                'random_state': 42,
                # Parámetros para optimizar memoria
                'verbose': -1,
                'max_bin': max_bin,
                'num_threads': 1 if max_memory_mb < 2000 else 2
            }
            model = LGBMRegressor(**params)
        else:
            raise ValueError(f"Modelo no soportado: {model_name}")
        
        # Evaluación con validación cruzada adaptada a memoria disponible
        try:
            # Reducir el número de folds si hay poca memoria
            n_folds = 3 if max_memory_mb < 2000 else 5
            from sklearn.model_selection import KFold
            
            kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
            scores = []
            
            for train_idx, val_idx in kf.split(X_train):
                X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
                y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
                
                # Entrenar modelo
                model.fit(X_fold_train, y_fold_train)
                
                # Evaluar y liberar memoria inmediatamente
                y_pred = model.predict(X_fold_val)
                rmse = np.sqrt(mean_squared_error(y_fold_val, y_pred))
                scores.append(rmse)
                
                # Limpieza agresiva de memoria
                del X_fold_train, X_fold_val, y_fold_train, y_fold_val, y_pred
                gc.collect()
            
            # Liberar modelo también
            del model
            gc.collect()
            
            return np.mean(scores)
        except Exception as e:
            print(f"Error en evaluación: {e}")
            return float('inf')
    
    return objective

def run_memory_efficient_optimization(model_name='XGBoost'):
    """Ejecuta optimización con Optuna adaptada a la memoria disponible"""
    print(f"\n🧠 Iniciando optimización adaptativa de memoria para {model_name}...")
    
    # 1. Detectar memoria disponible
    max_memory_mb = get_available_memory()
    
    # 2. Configurar Optuna según memoria disponible
    optuna_config = configure_optuna_for_memory_efficiency(max_memory_mb, model_name)
    
    # 3. Mostrar configuración
    print(f"\nConfiguración para {model_name}:")
    print(f"- Trials: {optuna_config['n_trials']}")
    print(f"- Paralelismo: {optuna_config['n_jobs']} jobs")
    print(f"- Almacenamiento: {'SQLite' if optuna_config['storage'] else 'En memoria'}")
    print(f"- Configuración específica: {optuna_config['model_config']}")
    
    # 4. Crear estudio Optuna
    from optuna.pruners import MedianPruner
    pruner = MedianPruner(n_startup_trials=5, n_warmup_steps=5)
    
    study = optuna.create_study(
        direction="minimize",
        pruner=pruner,
        storage=optuna_config['storage'],
        study_name=f"{model_name}_memory_optimized",
        load_if_exists=True
    )
    
    # 5. Crear función objetivo eficiente en memoria
    objective = memory_efficient_objective_factory(
        model_name, 
        X_train_scaled, 
        y_train, 
        max_memory_mb,
        optuna_config['model_config']
    )
    
    # 6. Callback para liberar memoria periódicamente
    def gc_callback(study, trial):
        if trial.number % 3 == 0:  # Cada 3 trials
            gc.collect()
    
    # 7. Visualización inicial
    color_map = {
        'RandomForest': '#e6f3ff',  # Azul claro
        'XGBoost': '#fff0e6',      # Naranja claro
        'LightGBM': '#e6fff2'      # Verde claro
    }
    display(HTML(f'<div style="background-color:{color_map.get(model_name, "#f5f5f5")}; padding:10px; border-radius:5px;">' +
                f'<h3>🔍 Optimización adaptativa de memoria - {model_name}</h3>' +
                f'<div>Memoria máxima a utilizar: {max_memory_mb:.0f} MB</div>' +
                f'<div>Trials planeados: {optuna_config["n_trials"]}</div>' +
                f'</div>'))
    
    # 8. Iniciar optimización
    callback = OptimizationProgressCallback(
        optuna_config['n_trials'], f"{model_name} (RAM optimizada)"
    )
    
    study.optimize(
        objective, 
        n_trials=optuna_config['n_trials'],
        n_jobs=optuna_config['n_jobs'],
        gc_after_trial=True,
        callbacks=[callback, gc_callback]
    )
    
    # 9. Obtener mejores parámetros
    best_params = study.best_params
    best_value = study.best_value
    
    # 10. Visualizar resultados
    display(HTML(f'<div style="background-color:{color_map.get(model_name, "#f5f5f5")}; padding:10px; border-radius:5px;">' +
                f'<h3>✅ Optimización completada - {model_name}</h3>' +
                f'<div><b>Mejor RMSE:</b> {best_value:.4f}</div>' +
                f'<div><b>Mejores parámetros:</b> {str(best_params)}</div>' +
                f'</div>'))
    
    # 11. Entrenar modelo final con los mejores parámetros
    print("\nEntrenando modelo final con los mejores parámetros...")
    
    if model_name == 'RandomForest':
        best_model = RandomForestRegressor(**best_params, random_state=42)
    elif model_name == 'XGBoost':
        best_model = XGBRegressor(**best_params, random_state=42)
    elif model_name == 'LightGBM':
        best_model = LGBMRegressor(**best_params, random_state=42)
    
    # 12. Evaluar modelo final
    better_model, metrics = entrenar_y_evaluar_modelo(
        best_model, f'{model_name}_RAM_Opt', X_train_scaled, y_train, X_test_scaled, y_test
    )
    
    # 13. Guardar resultados
    resultados_base[f'{model_name}_RAM_Opt'] = metrics
    modelo_file = guardar_modelo(better_model, f'{model_name}_RAM_Opt')
    modelos_guardados[f'{model_name}_RAM_Opt'] = modelo_file
    
    return best_params, better_model, metrics

# Ejemplo de uso (descomenta para ejecutar):
"""
# Para ejecutar la optimización adaptativa solo de un modelo:
xgb_params, xgb_model, xgb_metrics = run_memory_efficient_optimization('XGBoost')

# Para ejecutar todos los modelos con optimización de memoria RAM:
print("\n🚀 Iniciando optimización adaptativa para todos los modelos...")

# Optimizar RandomForest con control de memoria adaptativo
rf_params, rf_model, rf_metrics = run_memory_efficient_optimization('RandomForest')
gc.collect()  # Liberar memoria entre modelos

# Optimizar XGBoost con control de memoria adaptativo
xgb_params, xgb_model, xgb_metrics = run_memory_efficient_optimization('XGBoost')
gc.collect()  # Liberar memoria entre modelos

# Optimizar LightGBM con control de memoria adaptativo
lgbm_params, lgbm_model, lgbm_metrics = run_memory_efficient_optimization('LightGBM')
gc.collect()  # Liberar memoria final
"""

# Ejecutar solo esta celda para iniciar la optimización adaptativa de memoria para XGBoost
print("\n🔍 Ejecutando optimización adaptativa de memoria RAM para XGBoost...")
xgb_params, xgb_model, xgb_metrics = run_memory_efficient_optimization('XGBoost')

In [ ]:
# Importaciones adicionales para Deep Learning
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, save_model, load_model
from tensorflow.keras.layers import (Dense, Dropout, Conv2D, Conv3D, ConvLSTM2D, BatchNormalization, 
                                   MaxPooling2D, Flatten, Input, concatenate, Reshape, TimeDistributed)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

print("TensorFlow versión:", tf.__version__)

# Configurar GPU si está disponible
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    print(f"GPU disponible: {physical_devices}")
    # Permitir crecimiento de memoria según sea necesario
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
else:
    print("No se detectó GPU. Usando CPU.")

In [ ]:
# 1. Importaciones necesarias
import numpy as np
import pandas as pd
import xarray as xr
import optuna
import pickle
import datetime
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# Importaciones para barras de progreso y mejora de visualización
from tqdm.notebook import tqdm, trange
from IPython.display import display, HTML, clear_output
import time

# Configurar visualización más atractiva
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook", font_scale=1.2)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [20]:
# 2. Cargar el dataset NetCDF
def load_dataset(file_path):
    """Carga un archivo NetCDF y lo convierte a pandas DataFrame"""
    try:
        # Cargar el archivo NetCDF con xarray
        print(f"Intentando cargar el archivo: {file_path}")
        ds = xr.open_dataset(file_path)
        print("Archivo cargado exitosamente con xarray")

        # Mostrar información del dataset cargado
        print("\nInformación del dataset:")
        print(ds.info())
        print("\nVariables disponibles:")
        for var_name in ds.data_vars:
            print(f"- {var_name}: {ds[var_name].shape}")

        # Convertir a DataFrame
        df = ds.to_dataframe().reset_index()
        return df, ds
    except Exception as e:
        print(f"Error al cargar el archivo NetCDF: {e}")
        return None, None

# Ruta al dataset
data_file = BASE_PATH / 'data' / 'output' / 'complete_dataset_with_features.nc'
print(f"Buscando archivo en: {data_file}")

# Cargar el dataset
df, ds_original = load_dataset(data_file)

# Verificar si se cargó correctamente
if df is not None:
    print(f"Dataset cargado con éxito. Dimensiones: {df.shape}")
    print("\nPrimeras filas del DataFrame:")
    display(df.head())
else:
    print("No se pudo cargar el dataset. Verificar la ruta y el formato del archivo.")

Buscando archivo en: /content/drive/MyDrive/ml_precipitation_prediction/data/output/complete_dataset_with_features.nc
Intentando cargar el archivo: /content/drive/MyDrive/ml_precipitation_prediction/data/output/complete_dataset_with_features.nc
Archivo cargado exitosamente con xarray

Información del dataset:
xarray.Dataset {
dimensions:
	time = 530 ;
	latitude = 62 ;
	longitude = 66 ;

variables:
	datetime64[ns] time(time) ;
	float32 latitude(latitude) ;
	float32 longitude(longitude) ;
	float32 total_precipitation(time, latitude, longitude) ;
	float32 max_daily_precipitation(time, latitude, longitude) ;
	float32 min_daily_precipitation(time, latitude, longitude) ;
	float32 daily_precipitation_std(time, latitude, longitude) ;
	float32 month_sin(time, latitude, longitude) ;
	float32 month_cos(time, latitude, longitude) ;
	float32 doy_sin(time, latitude, longitude) ;
	float32 doy_cos(time, latitude, longitude) ;
	float64 elevation(latitude, longitude) ;
	float32 slope(latitude, longitude

,time,latitude,longitude,total_precipitation,max_daily_precipitation,min_daily_precipitation,daily_precipitation_std,month_sin,month_cos,doy_sin,doy_cos,elevation,slope,aspect
0,1981-01-01,4.324997,-74.975006,47.381050,24.706928,0.0,5.825776,0.5,0.866025,0.017202,0.999852,493.784552,89.539551,102.044502
1,1981-01-01,4.324997,-74.925003,40.750824,21.819195,0.0,5.019045,0.5,0.866025,0.017202,0.999852,519.750107,89.867020,73.481674
2,1981-01-01,4.324997,-74.875008,46.338623,26.092327,0.0,5.740223,0.5,0.866025,0.017202,0.999852,248.776045,89.722221,65.916817
3,1981-01-01,4.324997,-74.825005,48.779938,29.421450,0.0,5.611738,0.5,0.866025,0.017202,0.999852,351.415728,86.986130,140.916000
4,1981-01-01,4.324997,-74.775002,38.932945,18.483061,0.0,3.733574,0.5,0.866025,0.017202,0.999852,278.261922,88.273293,18.439939


In [21]:
# 3. Preparación de los datos
if df is not None:
    # Identificar la columna objetivo (precipitación)
    target_column = 'total_precipitation'  # Ajustar si tiene otro nombre en tu dataset

    # Ver si existe 'precip_target' o usar 'total_precipitation'
    if 'total_precipitation' in df.columns:
        target_column = 'total_precipitation'

    print(f"Columna objetivo identificada: {target_column}")

    # Separar variables predictoras y variable objetivo
    feature_cols = [col for col in df.columns if col != target_column and not pd.isna(df[col]).all()]

    # Eliminar columnas no numéricas para los modelos (como fechas o coordenadas si no se usan como features)
    non_feature_cols = ['time', 'spatial_ref']
    feature_cols = [col for col in feature_cols if col not in non_feature_cols]

    # Eliminar filas con valores NaN
    print(f"Filas antes de eliminar NaN: {df.shape[0]}")
    df_clean = df.dropna(subset=[target_column] + feature_cols)
    print(f"Filas después de eliminar NaN: {df_clean.shape[0]}")

    # Separar features y target
    X = df_clean[feature_cols]
    y = df_clean[target_column]

    print(f"\nFeatures seleccionadas ({len(feature_cols)}):\n{feature_cols}")
    print(f"\nVariable objetivo: {target_column}")

Columna objetivo identificada: total_precipitation
Filas antes de eliminar NaN: 2168760
Filas después de eliminar NaN: 2168760

Features seleccionadas (12):
['latitude', 'longitude', 'max_daily_precipitation', 'min_daily_precipitation', 'daily_precipitation_std', 'month_sin', 'month_cos', 'doy_sin', 'doy_cos', 'elevation', 'slope', 'aspect']

Variable objetivo: total_precipitation


In [22]:
# 4. División del conjunto de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Dimensiones del conjunto de entrenamiento: {X_train.shape}")
print(f"Dimensiones del conjunto de prueba: {X_test.shape}")

# 5. Estandarización de variables predictoras
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Guardar el scaler para uso futuro
with open(model_output_dir / 'scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print("Escalador guardado en models/output/scaler.pkl")

Dimensiones del conjunto de entrenamiento: (1735008, 12)
Dimensiones del conjunto de prueba: (433752, 12)
Escalador guardado en models/output/scaler.pkl


In [ ]:
# 6. Funciones de evaluación y entrenamiento
def evaluar_modelo(y_true, y_pred):
    """Evalúa el rendimiento de un modelo usando múltiples métricas"""
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return rmse, mae, r2

def entrenar_y_evaluar_modelo(modelo, nombre, X_train, y_train, X_test, y_test):
    """Entrena un modelo y evalúa su rendimiento con visualización del progreso"""
    # Crear widget para mostrar información del proceso
    display(HTML(f'<div style="background-color:#f0f8ff; padding:10px; border-radius:5px;">' +
                 f'<h3>🔄 Entrenando modelo: {nombre}</h3>' +
                 f'<div id="status_{nombre}">Estado: Iniciando entrenamiento...</div>' +
                 f'</div>'))
    
    # Tiempo de inicio
    start_time = time.time()
    
    # Entrenar el modelo con seguimiento visual según el tipo
    if hasattr(modelo, 'fit_generator') or nombre in ['XGBoost', 'XGBoost_Optuna', 'LightGBM', 'LightGBM_Optuna']:
        # Para modelos que soportan entrenamiento por lotes como XGBoost, LightGBM
        print(f"Entrenando {nombre} con visualización de progreso...")
        if hasattr(modelo, 'n_estimators'):
            n_estimators = modelo.n_estimators
            for i in tqdm(range(n_estimators), desc=f"Entrenando {nombre}"):
                if i == 0:
                    # Primera iteración, ajuste inicial
                    if nombre.startswith('LightGBM'):
                        # LightGBM tiene parámetro verbose
                        temp_modelo = type(modelo)(n_estimators=1, **{k:v for k,v in modelo.get_params().items() 
                                                                 if k != 'n_estimators' and k != 'verbose'}, verbose=-1)
                    else:
                        temp_modelo = type(modelo)(n_estimators=1, **{k:v for k,v in modelo.get_params().items() 
                                                                if k != 'n_estimators'})
                    temp_modelo.fit(X_train, y_train)
                elif i == n_estimators - 1:
                    # Última iteración, ajuste completo
                    modelo.fit(X_train, y_train)
                
                # Actualizar progreso visual
                if i % max(1, n_estimators // 10) == 0:
                    clear_output(wait=True)
                    display(HTML(f'<div style="background-color:#f0f8ff; padding:10px; border-radius:5px;">' +
                                f'<h3>🔄 Entrenando modelo: {nombre}</h3>' +
                                f'<div id="status_{nombre}">Estado: Progreso {i+1}/{n_estimators} estimadores ({((i+1)/n_estimators*100):.1f}%)</div>' +
                                f'</div>'))
                    time.sleep(0.1)  # Pequeña pausa para actualización visual
        else:
            # Si no tiene n_estimators, entrenamiento directo
            modelo.fit(X_train, y_train)
    else:
        # Para modelos estándar como RandomForest
        modelo.fit(X_train, y_train)
    
    # Tiempo de entrenamiento
    training_time = time.time() - start_time
    
    # Visualizar tiempo de entrenamiento
    display(HTML(f'<div style="background-color:#e6ffe6; padding:10px; border-radius:5px;">' +
                f'<h3>✅ Entrenamiento completado: {nombre}</h3>' +
                f'<div>Tiempo de entrenamiento: {training_time:.2f} segundos</div>' +
                f'</div>'))
    
    print(f"Evaluando rendimiento de {nombre}...")
    predicciones = modelo.predict(X_test)
    rmse, mae, r2 = evaluar_modelo(y_test, predicciones)
    
    # Visualizar métricas con estilo
    display(HTML(f'<div style="background-color:#f5f5dc; padding:10px; border-radius:5px; margin-top:10px;">' +
                f'<h3>📊 Métricas para {nombre}</h3>' +
                f'<table style="width:100%; text-align:left;">' +
                f'<tr><th>Métrica</th><th>Valor</th></tr>' +
                f'<tr><td>RMSE</td><td>{rmse:.4f}</td></tr>' +
                f'<tr><td>MAE</td><td>{mae:.4f}</td></tr>' +
                f'<tr><td>R²</td><td>{r2:.4f}</td></tr>' +
                f'</table></div>'))
    
    return modelo, (rmse, mae, r2)

def guardar_modelo(modelo, nombre):
    """Guarda un modelo entrenado en disco"""
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{nombre}_{timestamp}.pkl"
    with open(model_output_dir / filename, 'wb') as f:
        pickle.dump(modelo, f)
    
    # Visualizar confirmación de guardado
    display(HTML(f'<div style="background-color:#e6ffee; padding:10px; border-radius:5px; margin-top:10px;">' +
                f'<h3>💾 Modelo guardado</h3>' +
                f'<div>Modelo <b>{nombre}</b> guardado como: {filename}</div>' +
                f'</div>'))
    return filename

In [ ]:
# 7. Entrenamiento de modelos base sin optimización

# Inicializar diccionarios para almacenar resultados y modelos
resultados_base = {}  # Para almacenar métricas (RMSE, MAE, R2)
modelos_base = {}     # Para almacenar instancias de modelos
modelos_guardados = {} # Para almacenar nombres de archivos guardados

print("\n🔍 Entrenando modelos baseline sin optimización de hiperparámetros...")

# 1. Modelo RandomForest básico
print("\nEntrenando RandomForest base...")
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model, rf_metrics = entrenar_y_evaluar_modelo(
    rf_model, 'RandomForest', X_train_scaled, y_train, X_test_scaled, y_test
)
resultados_base['RandomForest'] = rf_metrics
modelos_base['RandomForest'] = rf_model
modelo_file = guardar_modelo(rf_model, 'RandomForest')
modelos_guardados['RandomForest'] = modelo_file

# Visualizar importancia de características para RandomForest
plt.figure(figsize=(12, 6))
feat_importances = rf_model.feature_importances_
indices = np.argsort(feat_importances)[::-1]
plt.bar(range(len(indices)), feat_importances[indices], color='skyblue')
plt.xticks(range(len(indices)), [feature_cols[i] for i in indices], rotation=90)
plt.title('Importancia de Características - RandomForest')
plt.tight_layout()
plt.savefig(model_output_dir / 'randomforest_feature_importance.png')
plt.show()

# 2. Modelo XGBoost básico
print("\nEntrenando XGBoost base...")
xgb_model = XGBRegressor(n_estimators=100, random_state=42)
xgb_model, xgb_metrics = entrenar_y_evaluar_modelo(
    xgb_model, 'XGBoost', X_train_scaled, y_train, X_test_scaled, y_test
)
resultados_base['XGBoost'] = xgb_metrics
modelos_base['XGBoost'] = xgb_model
modelo_file = guardar_modelo(xgb_model, 'XGBoost')
modelos_guardados['XGBoost'] = modelo_file

# Visualizar importancia de características para XGBoost
plt.figure(figsize=(12, 6))
feat_importances = xgb_model.feature_importances_
indices = np.argsort(feat_importances)[::-1]
plt.bar(range(len(indices)), feat_importances[indices], color='coral')
plt.xticks(range(len(indices)), [feature_cols[i] for i in indices], rotation=90)
plt.title('Importancia de Características - XGBoost')
plt.tight_layout()
plt.savefig(model_output_dir / 'xgboost_feature_importance.png')
plt.show()

# 3. Modelo LightGBM básico
print("\nEntrenando LightGBM base...")
lgbm_model = LGBMRegressor(n_estimators=100, random_state=42)
lgbm_model, lgbm_metrics = entrenar_y_evaluar_modelo(
    lgbm_model, 'LightGBM', X_train_scaled, y_train, X_test_scaled, y_test
)
resultados_base['LightGBM'] = lgbm_metrics
modelos_base['LightGBM'] = lgbm_model
modelo_file = guardar_modelo(lgbm_model, 'LightGBM')
modelos_guardados['LightGBM'] = modelo_file

# Visualizar importancia de características para LightGBM
plt.figure(figsize=(12, 6))
feat_importances = lgbm_model.feature_importances_
indices = np.argsort(feat_importances)[::-1]
plt.bar(range(len(indices)), feat_importances[indices], color='lightgreen')
plt.xticks(range(len(indices)), [feature_cols[i] for i in indices], rotation=90)
plt.title('Importancia de Características - LightGBM')
plt.tight_layout()
plt.savefig(model_output_dir / 'lightgbm_feature_importance.png')
plt.show()

# Comparar resultados de modelos base
print("\n🔍 Comparación de modelos base sin optimización:")
temp_df = pd.DataFrame(resultados_base, index=['RMSE', 'MAE', 'R2']).T
print("\nOrdenados por RMSE (menor es mejor):")
display(temp_df.sort_values('RMSE'))

# Visualizar comparación de RMSE
plt.figure(figsize=(10, 5))
sns.barplot(x=temp_df.index, y=temp_df['RMSE'])
plt.title('Comparación de RMSE - Modelos Base')
plt.ylabel('RMSE (menor es mejor)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(model_output_dir / 'baseline_rmse_comparison.png')
plt.show()

In [ ]:
# 8. Optimización de hiperparámetros con Optuna (Refactorizado para los tres modelos)

# Clase para mostrar el progreso de la optimización con Optuna
class OptimizationProgressCallback:
    def __init__(self, n_trials, desc="Optimización"):
        self.pbar = tqdm(total=n_trials, desc=desc)
        self.best_value = float('inf')
        self.n_trials = n_trials
        self.start_time = time.time()
        
    def __call__(self, study, trial):
        # Actualizar la mejor métrica si hay mejora
        if study.best_value < self.best_value:
            self.best_value = study.best_value
            best_params_str = ", ".join([f"{k}={v}" for k, v in study.best_params.items()])
            elapsed = time.time() - self.start_time
            minutes, seconds = divmod(elapsed, 60)
            clear_output(wait=True)
            display(HTML(f'<div style="background-color:#e6f3ff; padding:10px; border-radius:5px;">' +
                        f'<h3>🔍 Optimización de hiperparámetros en progreso</h3>' +
                        f'<div><b>Mejor RMSE:</b> {self.best_value:.4f}</div>' +
                        f'<div><b>Mejores parámetros:</b> {best_params_str}</div>' +
                        f'<div><b>Prueba actual:</b> {trial.number+1}/{self.n_trials}</div>' +
                        f'<div><b>Tiempo transcurrido:</b> {int(minutes)}m {int(seconds)}s</div>' +
                        f'<div><b>Progreso:</b></div>' +
                        f'</div>'))
        
        # Actualizar la barra de progreso
        self.pbar.update(1)
        self.pbar.set_postfix({"mejor_rmse": f"{study.best_value:.4f}"})
        
        # Si es la última iteración, cerramos la barra
        if trial.number + 1 == self.n_trials:
            self.pbar.close()

def optimize_model_hyperparams(model_name, n_trials=30):
    """Función para optimizar hiperparámetros de diferentes modelos usando Optuna"""
    # Definir espacio de búsqueda según el modelo
    def objective(trial):
        # Configuración de parámetros específicos para cada modelo
        if model_name == 'RandomForest':
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 500),
                'max_depth': trial.suggest_int('max_depth', 5, 50),
                'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
                'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
                'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
                'random_state': 42
            }
            model = RandomForestRegressor(**params)
        
        elif model_name == 'XGBoost':
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
                'max_depth': trial.suggest_int('max_depth', 3, 12),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                'subsample': trial.suggest_float('subsample', 0.5, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
                'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
                'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),
                'random_state': 42
            }
            model = XGBRegressor(**params)
        
        elif model_name == 'LightGBM':
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
                'num_leaves': trial.suggest_int('num_leaves', 20, 150),
                'max_depth': trial.suggest_int('max_depth', -1, 15),  # -1 significa sin restricción
                'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 5, 100),
                'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
                'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
                'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
                'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),
                'random_state': 42
            }
            model = LGBMRegressor(**params)
        else:
            raise ValueError(f"Modelo no soportado: {model_name}")
        
        try:
            score = cross_val_score(model, X_train_scaled, y_train,
                              scoring='neg_root_mean_squared_error', cv=5, n_jobs=-1,
                              error_score='raise')
            return -np.mean(score)  # Devolvemos negativo porque optimizamos minimizando
        except Exception as e:
            print(f"Error en prueba de hiperparámetros para {model_name}: {e}")
            return float('inf')
    
    # Crear y configurar estudio de Optuna
    print(f"\nIniciando optimización de hiperparámetros para {model_name}...")
    
    # Definir colores para cada modelo
    color_map = {
        'RandomForest': '#e6f3ff',  # Azul claro
        'XGBoost': '#fff0e6',      # Naranja claro
        'LightGBM': '#e6fff2'      # Verde claro
    }
    
    # Visualización inicial
    display(HTML(f'<div style="background-color:{color_map[model_name]}; padding:10px; border-radius:5px;">' +
                f'<h3>🔍 Iniciando optimización de hiperparámetros - {model_name}</h3>' +
                f'<div>Pruebas totales: {n_trials}</div>' +
                f'<div>Métrica objetivo: RMSE (menor es mejor)</div>' +
                f'</div>'))
    
    study = optuna.create_study(direction='minimize')
    callback = OptimizationProgressCallback(n_trials, f"{model_name} Optimization")
    exceptions = (ValueError,) if model_name == 'RandomForest' else None
    study.optimize(objective, n_trials=n_trials, catch=exceptions, callbacks=[callback])
    
    # Visualizar resultados finales de optimización
    best_rmse = study.best_value
    best_params = study.best_params
    best_params_str = ", ".join([f"{k}={v}" for k, v in best_params.items()])
    
    display(HTML(f'<div style="background-color:{color_map[model_name]}; padding:10px; border-radius:5px; margin-top:10px;">' +
                f'<h3>✅ Optimización completada - {model_name}</h3>' +
                f'<div><b>Mejor RMSE:</b> {best_rmse:.4f}</div>' +
                f'<div><b>Mejores parámetros:</b> {best_params_str}</div>' +
                f'</div>'))
    
    # Crear y entrenar el mejor modelo con los parámetros optimizados
    if model_name == 'RandomForest':
        best_model = RandomForestRegressor(**best_params, random_state=42)
    elif model_name == 'XGBoost':
        best_model = XGBRegressor(**best_params, random_state=42)
    elif model_name == 'LightGBM':
        best_model = LGBMRegressor(**best_params, random_state=42)
    
    # Entrenar y evaluar el modelo optimizado
    mejor_modelo, metricas = entrenar_y_evaluar_modelo(
        best_model, f'{model_name}_Optuna', X_train_scaled, y_train, X_test_scaled, y_test
    )
    
    # Guardar los resultados y el modelo
    resultados_base[f'{model_name}_Optuna'] = metricas
    modelo_file = guardar_modelo(mejor_modelo, f'{model_name}_Optuna')
    modelos_guardados[f'{model_name}_Optuna'] = modelo_file
    
    # Visualizar importancia de características
    feature_importances = mejor_modelo.feature_importances_
    indices = np.argsort(feature_importances)[::-1]
    
    # Definir colores para gráficas por modelo
    plot_color_map = {
        'RandomForest': 'skyblue',
        'XGBoost': 'coral',
        'LightGBM': 'lightgreen'
    }
    
    plt.figure(figsize=(12, 6))
    plt.bar(range(len(indices)), feature_importances[indices], color=plot_color_map[model_name])
    plt.xticks(range(len(indices)), [feature_cols[i] for i in indices], rotation=90)
    plt.title(f'Importancia de Características - {model_name} Optimizado')
    plt.tight_layout()
    plt.savefig(model_output_dir / f'{model_name.lower()}_feature_importance.png')
    plt.show()
    
    return best_params, mejor_modelo, metricas

# Optimizar cada uno de los modelos
n_trials_per_model = 30  # Número de pruebas para cada modelo

# Optimizar RandomForest
print("Iniciando optimización de hiperparámetros para RandomForest...")
rf_best_params, mejor_rf, rf_metrics = optimize_model_hyperparams('RandomForest', n_trials_per_model)

# Optimizar XGBoost
print("Iniciando optimización de hiperparámetros para XGBoost...")
xgb_best_params, mejor_xgb, xgb_metrics = optimize_model_hyperparams('XGBoost', n_trials_per_model)

# Optimizar LightGBM
print("Iniciando optimización de hiperparámetros para LightGBM...")
lgbm_best_params, mejor_lgbm, lgbm_metrics = optimize_model_hyperparams('LightGBM', n_trials_per_model)

[I 2025-04-28 01:29:40,112] A new study created in memory with name: no-name-f2097522-b917-45e3-a523-4d5b98a66044


Iniciando optimización de hiperparámetros para RandomForest...


[I 2025-04-28 01:58:10,879] Trial 0 finished with value: 36.8066452846521 and parameters: {'n_estimators': 343, 'max_depth': 34, 'min_samples_split': 5, 'min_samples_leaf': 10, 'max_features': 'log2'}. Best is trial 0 with value: 36.8066452846521.
[I 2025-04-28 02:18:38,335] Trial 1 finished with value: 41.63606143990411 and parameters: {'n_estimators': 304, 'max_depth': 12, 'min_samples_split': 4, 'min_samples_leaf': 10, 'max_features': 'log2'}. Best is trial 0 with value: 36.8066452846521.
[W 2025-04-28 02:18:39,457] Trial 2 failed with parameters: {'n_estimators': 371, 'max_depth': 49, 'min_samples_split': 7, 'min_samples_leaf': 10, 'max_features': 'auto'} because of the following error: ValueError('\nAll the 5 fits failed.\nIt is very likely that your model is misconfigured.\nYou can try to debug the error by setting error_score=\'raise\'.\n\nBelow are more details about the failures:\n--------------------------------------------------------------------------------\n3 fits failed w

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.11/dist-packages/sklearn/utils/_param_validation.py", line 98, in validate_parameter_constraints
    raise InvalidParameterError(
sklearn.utils._param_validation.InvalidParameterError: The 'max_feat

In [ ]:
# 9. Visualización de resultados
resultados_df = pd.DataFrame(resultados_base, index=['RMSE', 'MAE', 'R2']).T

# Visualizar resultados en tabla
display(resultados_df)

# Gráficas de comparación
plt.figure(figsize=(12, 6))
sns.barplot(x=resultados_df.index, y=resultados_df['RMSE'])
plt.title('Comparación de RMSE entre modelos')
plt.ylabel('RMSE (menor es mejor)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(model_output_dir / 'rmse_comparison.png')
plt.show()

# Gráfica de R²
plt.figure(figsize=(12, 6))
sns.barplot(x=resultados_df.index, y=resultados_df['R2'])
plt.title('Comparación de R² entre modelos')
plt.ylabel('R² (mayor es mejor)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(model_output_dir / 'r2_comparison.png')
plt.show()

In [ ]:
# 10. Exportar resultados finales
# Crear un dataset de xarray con resultados
def create_results_dataset(resultados_df):
    """Crea un dataset de xarray con los resultados para exportar como NetCDF"""
    modelo_names = list(resultados_df.index)
    metrics = ['RMSE', 'MAE', 'R2']

    # Crear arrays para cada métrica
    rmse_values = resultados_df['RMSE'].values
    mae_values = resultados_df['MAE'].values
    r2_values = resultados_df['R2'].values

    # Crear dataset
    ds = xr.Dataset(
        data_vars={
            'RMSE': (['model'], rmse_values),
            'MAE': (['model'], mae_values),
            'R2': (['model'], r2_values)
        },
        coords={
            'model': modelo_names,
        },
        attrs={
            'description': 'Resultados de modelos de predicción de precipitación STHyMOUNTAIN',
            'created': datetime.datetime.now().isoformat(),
            'features_used': ', '.join(feature_cols)
        }
    )
    return ds

# Crear dataset de resultados
results_ds = create_results_dataset(resultados_df)

# Guardar resultados como NetCDF
results_file = model_output_dir / 'model_results.nc'
results_ds.to_netcdf(results_file)
print(f"Resultados guardados como NetCDF en: {results_file}")

# También guardar como CSV para fácil visualización
csv_file = model_output_dir / 'model_results.csv'
resultados_df.to_csv(csv_file)
print(f"Resultados guardados como CSV en: {csv_file}")

# Crear un diccionario con información del modelo para uso futuro
model_info = {
    'date_trained': datetime.datetime.now().isoformat(),
    'feature_columns': feature_cols,
    'target_column': target_column,
    'models_saved': modelos_guardados,
    'results': resultados_df.to_dict(),
    'best_model': resultados_df['RMSE'].idxmin(),
    'scaler': 'scaler.pkl'
}

# Guardar información del modelo
with open(model_output_dir / 'model_info.pkl', 'wb') as f:
    pickle.dump(model_info, f)
print(f"Información del modelo guardada en: {model_output_dir / 'model_info.pkl'}")

print("\n🔥 Entrenamiento completado con éxito! El mejor modelo es:", resultados_df['RMSE'].idxmin())

In [ ]:
# 11. Preparación de datos para modelos espaciales y espaciotemporales

def reorganize_data_for_dl(df_clean, ds_original, target_column, feature_cols, time_steps=3):
    """
    Reorganiza los datos del DataFrame en formato espacial o espaciotemporal para modelos CNN y ConvLSTM
    
    Args:
        df_clean: DataFrame con datos limpios
        ds_original: Dataset xarray original
        target_column: Nombre de la columna objetivo
        feature_cols: Lista de columnas de características
        time_steps: Número de pasos temporales para secuencias (ConvLSTM)
    
    Returns:
        Datos para CNN y ConvLSTM en formato adecuado
    """
    print("Preparando datos para modelos espaciales y espaciotemporales...")
    
    # 1. Extraer información espacial del dataset original
    lats = ds_original.lat.values
    lons = ds_original.lon.values
    times = np.sort(df_clean['time'].unique())
    
    n_times = len(times)
    n_lats = len(lats)
    n_lons = len(lons)
    n_features = len(feature_cols)
    
    print(f"Dimensiones: tiempos={n_times}, lats={n_lats}, lons={n_lons}, features={n_features}")
    
    # Para CNN necesitamos: [samples, lat, lon, features]
    # Para ConvLSTM necesitamos: [samples, time_steps, lat, lon, features]
    
    # 2. Crear arrays vacíos para almacenar datos reestructurados
    # Para CNN (formato espacial)
    X_spatial = np.zeros((n_times, n_lats, n_lons, n_features))
    y_spatial = np.zeros((n_times, n_lats, n_lons, 1))
    
    # 3. Llenar arrays para CNN
    for i, t in enumerate(times):
        # Filtrar datos para este tiempo
        df_time = df_clean[df_clean['time'] == t]
        
        # Para cada lat/lon, extraer features y target
        for lat_idx, lat in enumerate(lats):
            for lon_idx, lon in enumerate(lons):
                # Obtener el registro para estas coordenadas
                df_point = df_time[(df_time['lat'] == lat) & (df_time['lon'] == lon)]
                
                if not df_point.empty:
                    # Extraer características
                    X_spatial[i, lat_idx, lon_idx, :] = df_point[feature_cols].values[0]
                    # Extraer target
                    y_spatial[i, lat_idx, lon_idx, 0] = df_point[target_column].values[0]
                
    print(f"Datos espaciales para CNN preparados. Forma X: {X_spatial.shape}, y: {y_spatial.shape}")
    
    # 4. Crear secuencias para ConvLSTM
    # Necesitamos [samples, time_steps, lat, lon, features]
    X_spatiotemporal = []
    y_spatiotemporal = []
    
    # Solo procesamos si hay suficientes pasos de tiempo
    if n_times > time_steps:
        for i in range(n_times - time_steps):
            # Secuencia de entrada: time_steps pasos consecutivos
            X_seq = X_spatial[i:i+time_steps, :, :, :]
            # Target: el valor del siguiente paso de tiempo
            y_seq = y_spatial[i+time_steps, :, :, :]
            
            X_spatiotemporal.append(X_seq)
            y_spatiotemporal.append(y_seq)
        
        # Convertir a arrays numpy
        X_spatiotemporal = np.array(X_spatiotemporal)
        y_spatiotemporal = np.array(y_spatiotemporal)
        
        print(f"Datos espaciotemporales para ConvLSTM preparados. Forma X: {X_spatiotemporal.shape}, y: {y_spatiotemporal.shape}")
        
        # División en entrenamiento/prueba para CNN
        X_train_cnn, X_test_cnn, y_train_cnn, y_test_cnn = train_test_split(
            X_spatial, y_spatial, test_size=0.2, random_state=42)
        
        # División en entrenamiento/prueba para ConvLSTM
        X_train_convlstm, X_test_convlstm, y_train_convlstm, y_test_convlstm = train_test_split(
            X_spatiotemporal, y_spatiotemporal, test_size=0.2, random_state=42)
        
        # Normalizar datos, solo las características
        # Para CNN
        for i in range(n_features):
            # Calcular media y desviación estándar en datos de entrenamiento
            feature_mean = np.mean(X_train_cnn[:, :, :, i])
            feature_std = np.std(X_train_cnn[:, :, :, i])
            
            # Normalizar train y test
            X_train_cnn[:, :, :, i] = (X_train_cnn[:, :, :, i] - feature_mean) / (feature_std + 1e-8)
            X_test_cnn[:, :, :, i] = (X_test_cnn[:, :, :, i] - feature_mean) / (feature_std + 1e-8)
        
        # Para ConvLSTM (normalizar cada característica en cada paso de tiempo)
        for i in range(n_features):
            for t in range(time_steps):
                # Calcular media y desviación estándar en datos de entrenamiento
                feature_mean = np.mean(X_train_convlstm[:, t, :, :, i])
                feature_std = np.std(X_train_convlstm[:, t, :, :, i])
                
                # Normalizar train y test
                X_train_convlstm[:, t, :, :, i] = (X_train_convlstm[:, t, :, :, i] - feature_mean) / (feature_std + 1e-8)
                X_test_convlstm[:, t, :, :, i] = (X_test_convlstm[:, t, :, :, i] - feature_mean) / (feature_std + 1e-8)
        
        return (
            (X_train_cnn, X_test_cnn, y_train_cnn, y_test_cnn),
            (X_train_convlstm, X_test_convlstm, y_train_convlstm, y_test_convlstm),
            (lats, lons, times)
        )
    else:
        print("No hay suficientes datos temporales para crear secuencias ConvLSTM.")
        # División en entrenamiento/prueba solo para CNN
        X_train_cnn, X_test_cnn, y_train_cnn, y_test_cnn = train_test_split(
            X_spatial, y_spatial, test_size=0.2, random_state=42)
        
        # Normalizar datos CNN
        for i in range(n_features):
            feature_mean = np.mean(X_train_cnn[:, :, :, i])
            feature_std = np.std(X_train_cnn[:, :, :, i])
            
            X_train_cnn[:, :, :, i] = (X_train_cnn[:, :, :, i] - feature_mean) / (feature_std + 1e-8)
            X_test_cnn[:, :, :, i] = (X_test_cnn[:, :, :, i] - feature_mean) / (feature_std + 1e-8)
        
        return (
            (X_train_cnn, X_test_cnn, y_train_cnn, y_test_cnn),
            None,
            (lats, lons, times)
        )

# Preparar datos para CNN y ConvLSTM
try:
    time_steps = 3  # Número de pasos temporales para secuencias
    spatial_data, spatiotemporal_data, spatial_coords = reorganize_data_for_dl(
        df_clean, ds_original, target_column, feature_cols, time_steps)
    
    # Desempaquetar datos espaciales (CNN)
    X_train_cnn, X_test_cnn, y_train_cnn, y_test_cnn = spatial_data
    
    # Verificar si hay datos espaciotemporales (ConvLSTM)
    if spatiotemporal_data is not None:
        X_train_convlstm, X_test_convlstm, y_train_convlstm, y_test_convlstm = spatiotemporal_data
        print("Datos para CNN y ConvLSTM preparados exitosamente.")
    else:
        print("Solo se prepararon datos para CNN. No hay suficientes datos temporales para ConvLSTM.")
    
    # Recuperar coordenadas espaciales
    lats, lons, times = spatial_coords
except Exception as e:
    print(f"Error al preparar datos para modelos espaciales: {e}")

In [ ]:
# 12. Implementación de modelo CNN para patrones espaciales

def create_cnn_model(input_shape):
    """
    Crea una arquitectura CNN para capturar patrones espaciales.
    
    Args:
        input_shape: Dimensiones de los datos de entrada (lat, lon, features)
    
    Returns:
        Modelo compilado de Keras
    """
    model = Sequential([
        # Primera capa convolucional
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        
        # Segunda capa convolucional
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        
        # Tercera capa convolucional
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        
        # Flatten para conectar a capas densas
        Flatten(),
        
        # Capas densas
        Dense(128, activation='relu'),
        Dropout(0.3),  # Regularización para prevenir sobreajuste
        Dense(64, activation='relu'),
        Dropout(0.2),
        
        # Capa de salida: Reshape para obtener predicciones espaciales
        Dense(np.prod(input_shape[:2])),
        Reshape(input_shape[:2] + (1,))  # Reshape a [lat, lon, 1]
    ])
    
    # Compilar modelo con optimizador Adam
    optimizer = Adam(learning_rate=0.001)
    model.compile(
        optimizer=optimizer,
        loss='mse',  # Error cuadrático medio para regresión
        metrics=['mae']  # Error absoluto medio como métrica adicional
    )
    
    print(f"Modelo CNN creado con input_shape={input_shape}")
    model.summary()
    
    return model

try:
    # Obtener dimensiones de entrada a partir de los datos de entrenamiento
    input_shape_cnn = X_train_cnn.shape[1:]  # [lat, lon, features]
    print(f"Forma de datos de entrada CNN: {input_shape_cnn}")
    
    # Crear y entrenar modelo CNN
    model_cnn = create_cnn_model(input_shape_cnn)
    
    # Callbacks para entrenamiento
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
        ModelCheckpoint(filepath=str(model_output_dir / 'cnn_model.h5'), 
                       save_best_only=True, monitor='val_loss')
    ]
    
    # Entrenar modelo
    print("\nEntrenando modelo CNN...")
    history_cnn = model_cnn.fit(
        X_train_cnn, 
        y_train_cnn,
        batch_size=16,
        epochs=50,
        validation_split=0.2,
        callbacks=callbacks,
        verbose=1
    )
    
    # Evaluar modelo en conjunto de prueba
    print("\nEvaluando modelo CNN en conjunto de prueba...")
    y_pred_cnn = model_cnn.predict(X_test_cnn)
    
    # Aplanar para evaluación
    y_test_flat = y_test_cnn.reshape(-1)
    y_pred_flat = y_pred_cnn.reshape(-1)
    
    # Eliminar valores NaN si existen
    mask = ~np.isnan(y_test_flat) & ~np.isnan(y_pred_flat)
    y_test_valid = y_test_flat[mask]
    y_pred_valid = y_pred_flat[mask]
    
    # Calcular métricas
    rmse_cnn = np.sqrt(mean_squared_error(y_test_valid, y_pred_valid))
    mae_cnn = mean_absolute_error(y_test_valid, y_pred_valid)
    r2_cnn = r2_score(y_test_valid, y_pred_valid)
    
    print(f"CNN - RMSE: {rmse_cnn:.4f}, MAE: {mae_cnn:.4f}, R²: {r2_cnn:.4f}")
    
    # Añadir resultados al DataFrame de resultados
    resultados_base['CNN'] = (rmse_cnn, mae_cnn, r2_cnn)
    
    # Guardar modelo en formato H5
    model_cnn_file = model_output_dir / 'cnn_model.h5'
    model_cnn.save(model_cnn_file)
    print(f"Modelo CNN guardado en: {model_cnn_file}")
    
    # Visualizar historia de entrenamiento
    plt.figure(figsize=(12, 5))
    
    # Gráfica de pérdida
    plt.subplot(1, 2, 1)
    plt.plot(history_cnn.history['loss'], label='Train')
    plt.plot(history_cnn.history['val_loss'], label='Validation')
    plt.title('Pérdida CNN')
    plt.xlabel('Epoch')
    plt.ylabel('Loss (MSE)')
    plt.legend()
    
    # Gráfica de MAE
    plt.subplot(1, 2, 2)
    plt.plot(history_cnn.history['mae'], label='Train')
    plt.plot(history_cnn.history['val_mae'], label='Validation')
    plt.title('MAE CNN')
    plt.xlabel('Epoch')
    plt.ylabel('MAE')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(model_output_dir / 'cnn_training_history.png')
    plt.show()
    
    # Crear visualización espacial del error
    # Elegir un ejemplo aleatorio del conjunto de prueba para visualización
    sample_idx = np.random.randint(0, len(X_test_cnn))
    sample_true = y_test_cnn[sample_idx, :, :, 0]
    sample_pred = y_pred_cnn[sample_idx, :, :, 0]
    sample_error = np.abs(sample_true - sample_pred)
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Mapa de precipitación real
    im0 = axes[0].imshow(sample_true, cmap='Blues')
    axes[0].set_title('Precipitación Real')
    plt.colorbar(im0, ax=axes[0], label='mm')
    
    # Mapa de precipitación predicha
    im1 = axes[1].imshow(sample_pred, cmap='Blues')
    axes[1].set_title('Precipitación Predicha (CNN)')
    plt.colorbar(im1, ax=axes[1], label='mm')
    
    # Mapa de error
    im2 = axes[2].imshow(sample_error, cmap='Reds')
    axes[2].set_title('Error Absoluto')
    plt.colorbar(im2, ax=axes[2], label='mm')
    
    plt.tight_layout()
    plt.savefig(model_output_dir / 'cnn_spatial_error.png')
    plt.show()
except Exception as e:
    print(f"Error al implementar modelo CNN: {e}")

In [ ]:
# 13. Implementación de modelo ConvLSTM para patrones espaciotemporales

def create_convlstm_model(input_shape):
    """
    Crea una arquitectura ConvLSTM para capturar patrones espaciotemporales.
    
    Args:
        input_shape: Dimensiones de los datos de entrada (time_steps, lat, lon, features)
    
    Returns:
        Modelo compilado de Keras
    """
    model = Sequential([
        # Primera capa ConvLSTM
        ConvLSTM2D(filters=32, kernel_size=(3, 3), padding='same',
                   return_sequences=True, activation='tanh',
                   recurrent_activation='hard_sigmoid',
                   input_shape=input_shape),
        BatchNormalization(),
        
        # Segunda capa ConvLSTM
        ConvLSTM2D(filters=64, kernel_size=(3, 3), padding='same',
                   return_sequences=False, activation='tanh',
                   recurrent_activation='hard_sigmoid'),
        BatchNormalization(),
        
        # Capas convolucionales para procesamiento final
        Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'),
        BatchNormalization(),
        
        # Capa de salida para predicción
        Conv2D(filters=1, kernel_size=(3, 3), padding='same', activation='linear')
    ])
    
    # Compilar modelo con optimizador Adam
    optimizer = Adam(learning_rate=0.001)
    model.compile(
        optimizer=optimizer,
        loss='mse',  # Error cuadrático medio para regresión
        metrics=['mae']  # Error absoluto medio como métrica adicional
    )
    
    print(f"Modelo ConvLSTM creado con input_shape={input_shape}")
    model.summary()
    
    return model

# Creamos una clase para visualizar el progreso de entrenamiento con tqdm
class TqdmCallback(tf.keras.callbacks.Callback):
    def __init__(self, epochs, metrics=['loss', 'val_loss']):
        self.epochs = epochs
        self.metrics = metrics
        self.tqdm_progress = None
        self.epoch_count = 0
        self.training_start = None
        
    def on_train_begin(self, logs=None):
        self.training_start = time.time()
        display(HTML(f'<div style="background-color:#e6f2ff; padding:10px; border-radius:5px;">' +
                     f'<h3>🧠 Iniciando entrenamiento de ConvLSTM</h3>' +
                     f'<div>Total épocas: {self.epochs}</div>' +
                     f'<div>Métricas monitorizadas: {", ".join(self.metrics)}</div>' +
                     f'</div>'))
        self.tqdm_progress = tqdm(total=self.epochs, desc="Entrenando ConvLSTM")
        
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epoch_count += 1
        
        # Recoger métricas para mostrar
        metrics_str = ", ".join([f"{m}: {logs.get(m, 0):.4f}" for m in self.metrics if m in logs])
        self.tqdm_progress.set_postfix_str(metrics_str)
        self.tqdm_progress.update(1)
        
        # Cada 5 épocas (o en la última), mostramos un resumen más completo
        if epoch % 5 == 0 or epoch == self.epochs - 1:
            elapsed = time.time() - self.training_start
            minutes, seconds = divmod(elapsed, 60)
            hours, minutes = divmod(minutes, 60)
            
            # Crear un resumen de progreso bonito
            clear_output(wait=True)
            display(HTML(f'<div style="background-color:#e6f2ff; padding:10px; border-radius:5px;">' +
                         f'<h3>🧠 Entrenando ConvLSTM - Progreso</h3>' +
                         f'<div><b>Época:</b> {epoch+1}/{self.epochs} ({((epoch+1)/self.epochs*100):.1f}%)</div>' +
                         f'<div><b>Tiempo transcurrido:</b> {int(hours)}h {int(minutes)}m {int(seconds)}s</div>' +
                         f'<div><b>Métricas actuales:</b> {metrics_str}</div>' +
                         f'<div><b>Mejor val_loss:</b> {min([logs.get("val_loss", float("inf"))] + [logs.get("val_loss", float("inf")) for logs in self.model.history.history.get("val_loss", []) or []]):.4f}</div>' +
                         f'</div>'))
            self.tqdm_progress = tqdm(total=self.epochs, initial=self.epoch_count, desc="Entrenando ConvLSTM")
            
    def on_train_end(self, logs=None):
        self.tqdm_progress.close()
        elapsed = time.time() - self.training_start
        minutes, seconds = divmod(elapsed, 60)
        hours, minutes = divmod(minutes, 60)
        
        display(HTML(f'<div style="background-color:#e6ffe6; padding:10px; border-radius:5px;">' +
                     f'<h3>✅ Entrenamiento de ConvLSTM completado</h3>' +
                     f'<div><b>Tiempo total:</b> {int(hours)}h {int(minutes)}m {int(seconds)}s</div>' +
                     f'</div>'))

try:
    # Verificar si tenemos datos espaciotemporales para ConvLSTM
    if 'spatiotemporal_data' in locals() and spatiotemporal_data is not None:
        # Crear y entrenar modelo ConvLSTM
        input_shape_convlstm = X_train_convlstm.shape[1:]  # [time_steps, lat, lon, features]
        print(f"Forma de datos de entrada ConvLSTM: {input_shape_convlstm}")
        
        # Crear modelo
        model_convlstm = create_convlstm_model(input_shape_convlstm)
        
        # Callbacks para entrenamiento
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        model_checkpoint = ModelCheckpoint(
            filepath=str(model_output_dir / 'convlstm_model.h5'),
            save_best_only=True,
            monitor='val_loss'
        )
        tqdm_callback = TqdmCallback(epochs=50, metrics=['loss', 'val_loss', 'mae', 'val_mae'])
        
        # Entrenar modelo con barra de progreso personalizada
        print("\nEntrenando modelo ConvLSTM...")
        history_convlstm = model_convlstm.fit(
            X_train_convlstm,
            y_train_convlstm,
            batch_size=16,
            epochs=50,
            validation_split=0.2,
            callbacks=[early_stopping, model_checkpoint, tqdm_callback],
            verbose=0  # No mostrar la barra de progreso estándar, usamos nuestra personalizada
        )
        
        # Evaluar modelo en conjunto de prueba
        print("\nEvaluando modelo ConvLSTM en conjunto de prueba...")
        y_pred_convlstm = model_convlstm.predict(X_test_convlstm)
        
        # Aplanar para evaluación
        y_test_flat_convlstm = y_test_convlstm.reshape(-1)
        y_pred_flat_convlstm = y_pred_convlstm.reshape(-1)
        
        # Eliminar valores NaN si existen
        mask_convlstm = ~np.isnan(y_test_flat_convlstm) & ~np.isnan(y_pred_flat_convlstm)
        y_test_valid_convlstm = y_test_flat_convlstm[mask_convlstm]
        y_pred_valid_convlstm = y_pred_flat_convlstm[mask_convlstm]
        
        # Calcular métricas
        rmse_convlstm = np.sqrt(mean_squared_error(y_test_valid_convlstm, y_pred_valid_convlstm))
        mae_convlstm = mean_absolute_error(y_test_valid_convlstm, y_pred_valid_convlstm)
        r2_convlstm = r2_score(y_test_valid_convlstm, y_pred_valid_convlstm)
        
        print(f"ConvLSTM - RMSE: {rmse_convlstm:.4f}, MAE: {mae_convlstm:.4f}, R²: {r2_convlstm:.4f}")
        
        # Visualizar resultados con estilo
        display(HTML(f'<div style="background-color:#f5f5dc; padding:10px; border-radius:5px; margin-top:10px;">' +
                    f'<h3>📊 Métricas para ConvLSTM</h3>' +
                    f'<table style="width:100%; text-align:left;">' +
                    f'<tr><th>Métrica</th><th>Valor</th></tr>' +
                    f'<tr><td>RMSE</td><td>{rmse_convlstm:.4f}</td></tr>' +
                    f'<tr><td>MAE</td><td>{mae_convlstm:.4f}</td></tr>' +
                    f'<tr><td>R²</td><td>{r2_convlstm:.4f}</td></tr>' +
                    f'</table></div>'))
        
        # Añadir resultados al DataFrame de resultados
        resultados_base['ConvLSTM'] = (rmse_convlstm, mae_convlstm, r2_convlstm)
        
        # Guardar modelo en formato H5
        model_convlstm_file = model_output_dir / 'convlstm_model.h5'
        model_convlstm.save(model_convlstm_file)
        print(f"Modelo ConvLSTM guardado en: {model_convlstm_file}")
        
        # Visualizar historia de entrenamiento
        plt.figure(figsize=(12, 5))
        
        # Gráfica de pérdida
        plt.subplot(1, 2, 1)
        plt.plot(history_convlstm.history['loss'], label='Train', linewidth=2)
        plt.plot(history_convlstm.history['val_loss'], label='Validation', linewidth=2, linestyle='--')
        plt.title('Pérdida ConvLSTM', fontsize=14)
        plt.xlabel('Epoch', fontsize=12)
        plt.ylabel('Loss (MSE)', fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.legend(fontsize=12)
        
        # Gráfica de MAE
        plt.subplot(1, 2, 2)
        plt.plot(history_convlstm.history['mae'], label='Train', linewidth=2)
        plt.plot(history_convlstm.history['val_mae'], label='Validation', linewidth=2, linestyle='--')
        plt.title('MAE ConvLSTM', fontsize=14)
        plt.xlabel('Epoch', fontsize=12)
        plt.ylabel('MAE', fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.legend(fontsize=12)
        
        plt.tight_layout()
        plt.savefig(model_output_dir / 'convlstm_training_history.png')
        plt.show()
        
        # Visualizar predicciones vs valores reales
        # Elegir un ejemplo aleatorio del conjunto de prueba para visualización
        sample_idx_convlstm = np.random.randint(0, len(X_test_convlstm))
        sample_true_convlstm = y_test_convlstm[sample_idx_convlstm, :, :, 0]
        sample_pred_convlstm = y_pred_convlstm[sample_idx_convlstm, :, :, 0]
        sample_error_convlstm = np.abs(sample_true_convlstm - sample_pred_convlstm)
        
        # Crear gráficas con mejor aspecto visual
        fig, axes = plt.subplots(1, 3, figsize=(18, 5))
        
        # Mapa de precipitación real
        im0 = axes[0].imshow(sample_true_convlstm, cmap='Blues', interpolation='nearest')
        axes[0].set_title('Precipitación Real', fontsize=14)
        plt.colorbar(im0, ax=axes[0], label='mm')
        axes[0].grid(False)
        
        # Mapa de precipitación predicha
        im1 = axes[1].imshow(sample_pred_convlstm, cmap='Blues', interpolation='nearest')
        axes[1].set_title('Predicción ConvLSTM', fontsize=14)
        plt.colorbar(im1, ax=axes[1], label='mm')
        axes[1].grid(False)
        
        # Mapa de error
        im2 = axes[2].imshow(sample_error_convlstm, cmap='Reds', interpolation='nearest')
        axes[2].set_title('Error Absoluto', fontsize=14)
        plt.colorbar(im2, ax=axes[2], label='mm')
        axes[2].grid(False)
        
        plt.tight_layout()
        plt.savefig(model_output_dir / 'convlstm_predictions.png')
        plt.show()
        
        # Resumen final de resultados
        print("\n🌟 Modelos entrenados y evaluados:")
        display(HTML(f'<div style="background-color:#f0f8ff; padding:15px; border-radius:8px; margin-top:20px;">' +
                    f'<h2>🏆 Resumen de Resultados</h2>' +
                    f'<p>Se han entrenado tanto modelos de machine learning (RandomForest, XGBoost, LightGBM) ' +
                    f'como modelos de deep learning (CNN, ConvLSTM).</p>' +
                    f'<h3>Ranking según RMSE (menor es mejor):</h3>' +
                    f'</div>'))
        
        # Ranking de modelos
        resultados_df_final = pd.DataFrame(resultados_base, index=['RMSE', 'MAE', 'R2']).T
        resultados_ordenados = resultados_df_final.sort_values('RMSE')
        display(resultados_ordenados)
        
        # Gráfica de resumen
        plt.figure(figsize=(14, 8))
        plt.subplot(2, 1, 1)
        sns.barplot(x=resultados_ordenados.index, y=resultados_ordenados['RMSE'], palette='viridis')
        plt.title('RMSE por Modelo (menor es mejor)', fontsize=14)
        plt.ylabel('RMSE (mm)', fontsize=12)
        plt.xticks(rotation=45, ha='right', fontsize=10)
        plt.grid(True, axis='y', alpha=0.3)
        
        plt.subplot(2, 1, 2)
        sns.barplot(x=resultados_ordenados.index, y=resultados_ordenados['R2'], palette='plasma')
        plt.title('R² por Modelo (mayor es mejor)', fontsize=14)
        plt.ylabel('Coeficiente R²', fontsize=12)
        plt.xticks(rotation=45, ha='right', fontsize=10)
        plt.grid(True, axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(model_output_dir / 'modelos_ranking_final.png')
        plt.show()
    else:
        print("No se pudieron preparar los datos espaciotemporales para el modelo ConvLSTM.")
except Exception as e:
    print(f"Error al implementar modelo ConvLSTM: {e}")

In [ ]:
# 14. Validación cruzada (Cross-Validation) para evaluar modelos

def evaluate_model_with_cv(modelo, nombre, X, y, n_splits=5):
    """
    Evalúa el modelo usando validación cruzada (k-fold)
    
    Args:
        modelo: Modelo a evaluar
        nombre: Nombre del modelo
        X: Características
        y: Variable objetivo
        n_splits: Número de divisiones (folds)
        
    Returns:
        Promedio de métricas y lista de métricas por fold
    """
    print(f"\nRealizando validación cruzada para {nombre} con {n_splits} folds...")
    
    # Crear instancia de KFold
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # Listas para almacenar métricas de cada fold
    rmse_scores = []
    mae_scores = []
    r2_scores = []
    
    # Iterar sobre cada fold
    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        print(f"Fold {fold+1}/{n_splits}")
        
        # Dividir datos para este fold
        X_train_fold, X_val_fold = X[train_idx], X[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
        
        # Entrenar modelo
        modelo.fit(X_train_fold, y_train_fold)
        
        # Predecir
        y_pred_fold = modelo.predict(X_val_fold)
        
        # Calcular métricas
        rmse = np.sqrt(mean_squared_error(y_val_fold, y_pred_fold))
        mae = mean_absolute_error(y_val_fold, y_pred_fold)
        r2 = r2_score(y_val_fold, y_pred_fold)
        
        # Almacenar métricas
        rmse_scores.append(rmse)
        mae_scores.append(mae)
        r2_scores.append(r2)
        
        print(f"Fold {fold+1} - RMSE: {rmse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")
    
    # Calcular promedios
    mean_rmse = np.mean(rmse_scores)
    mean_mae = np.mean(mae_scores)
    mean_r2 = np.mean(r2_scores)
    
    # Calcular desviaciones estándar
    std_rmse = np.std(rmse_scores)
    std_mae = np.std(mae_scores)
    std_r2 = np.std(r2_scores)
    
    print(f"\nResumen de validación cruzada para {nombre}:")
    print(f"RMSE: {mean_rmse:.4f} ± {std_rmse:.4f}")
    print(f"MAE: {mean_mae:.4f} ± {std_mae:.4f}")
    print(f"R²: {mean_r2:.4f} ± {std_r2:.4f}")
    
    # Crear diccionario con resultados
    cv_results = {
        'mean_rmse': mean_rmse,
        'mean_mae': mean_mae,
        'mean_r2': mean_r2,
        'std_rmse': std_rmse,
        'std_mae': std_mae,
        'std_r2': std_r2,
        'rmse_scores': rmse_scores,
        'mae_scores': mae_scores,
        'r2_scores': r2_scores
    }
    
    return cv_results

# Aplicar validación cruzada a los modelos tabulares
print("\n🔄 Ejecutando validación cruzada para modelos tabulares...")
cv_results = {}

# Lista de modelos a evaluar con validación cruzada
models_for_cv = {
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, random_state=42),
    'LightGBM': LGBMRegressor(n_estimators=100, random_state=42)
}

# Solo tomamos el mejor modelo de cada categoría para CV
if 'RandomForest_Optuna' in resultados_base:
    models_for_cv['RandomForest_Optuna'] = RandomForestRegressor(**rf_best_params, random_state=42)
if 'XGBoost_Optuna' in resultados_base:
    models_for_cv['XGBoost_Optuna'] = XGBRegressor(**xgb_best_params, random_state=42)
if 'LightGBM_Optuna' in resultados_base:
    models_for_cv['LightGBM_Optuna'] = LGBMRegressor(**lgbm_best_params, random_state=42)

# Ejecutar validación cruzada para cada modelo
for nombre, modelo in models_for_cv.items():
    cv_results[nombre] = evaluate_model_with_cv(modelo, nombre, X_train_scaled, y_train, n_splits=5)

# Visualizar resultados de validación cruzada
plt.figure(figsize=(12, 6))
modelo_names = list(cv_results.keys())
mean_rmses = [cv_results[model]['mean_rmse'] for model in modelo_names]
std_rmses = [cv_results[model]['std_rmse'] for model in modelo_names]

# Crear barplot con barras de error
bars = plt.bar(modelo_names, mean_rmses, yerr=std_rmses, capsize=5, color='skyblue', alpha=0.7)
plt.title('Comparación de RMSE con Validación Cruzada (5-fold)')
plt.ylabel('RMSE (menor es mejor)')
plt.xticks(rotation=45)

# Añadir valores en las barras
for bar, rmse, std in zip(bars, mean_rmses, std_rmses):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + std + 0.05,
             f'{rmse:.3f}±{std:.3f}', ha='center', va='bottom', rotation=0, size=9)

plt.tight_layout()
plt.savefig(model_output_dir / 'cross_validation_rmse.png')
plt.show()

In [ ]:
# 15. Bootstrapping para evaluar intervalos de confianza

def bootstrap_evaluate_model(model, X, y, n_iterations=100):
    """
    Evalúa el modelo usando bootstrapping para obtener intervalos de confianza
    
    Args:
        model: Modelo entrenado a evaluar
        X: Características
        y: Variable objetivo
        n_iterations: Número de iteraciones de bootstrap
    
    Returns:
        Diccionario con métricas e intervalos de confianza
    """
    print(f"Realizando bootstrapping con {n_iterations} iteraciones...")
    
    # Convertir a numpy arrays si no lo son ya
    if not isinstance(X, np.ndarray):
        X = X.values
    if not isinstance(y, np.ndarray):
        y = y.values
        
    n_samples = X.shape[0]
    
    # Listas para almacenar métricas de cada iteración
    rmse_scores = []
    mae_scores = []
    r2_scores = []
    
    # Realizar iteraciones de bootstrap
    for i in range(n_iterations):
        if i % 20 == 0:  # Solo imprimir cada 20 iteraciones para no saturar la salida
            print(f"Iteración bootstrap {i+1}/{n_iterations}")
            
        # Muestreo con reemplazo
        indices = np.random.choice(n_samples, n_samples, replace=True)
        
        # Crear muestra bootstrap
        X_boot = X[indices, :]
        y_boot = y[indices]
        
        # Predecir
        y_pred = model.predict(X_boot)
        
        # Calcular métricas
        rmse = np.sqrt(mean_squared_error(y_boot, y_pred))
        mae = mean_absolute_error(y_boot, y_pred)
        r2 = r2_score(y_boot, y_pred)
        
        # Almacenar métricas
        rmse_scores.append(rmse)
        mae_scores.append(mae)
        r2_scores.append(r2)
    
    # Calcular estadísticas de bootstrapping
    # Mediana como estimación central y percentiles 2.5 y 97.5 para intervalo de confianza del 95%
    rmse_median = np.median(rmse_scores)
    rmse_lower = np.percentile(rmse_scores, 2.5)
    rmse_upper = np.percentile(rmse_scores, 97.5)
    
    mae_median = np.median(mae_scores)
    mae_lower = np.percentile(mae_scores, 2.5)
    mae_upper = np.percentile(mae_scores, 97.5)
    
    r2_median = np.median(r2_scores)
    r2_lower = np.percentile(r2_scores, 2.5)
    r2_upper = np.percentile(r2_scores, 97.5)
    
    print("\nResultados de bootstrapping:")
    print(f"RMSE: {rmse_median:.4f} (IC 95%: [{rmse_lower:.4f}, {rmse_upper:.4f}])")
    print(f"MAE: {mae_median:.4f} (IC 95%: [{mae_lower:.4f}, {mae_upper:.4f}])")
    print(f"R²: {r2_median:.4f} (IC 95%: [{r2_lower:.4f}, {r2_upper:.4f}])")
    
    # Crear diccionario con resultados
    bootstrap_results = {
        'rmse_median': rmse_median,
        'rmse_ci': (rmse_lower, rmse_upper),
        'mae_median': mae_median,
        'mae_ci': (mae_lower, mae_upper),
        'r2_median': r2_median,
        'r2_ci': (r2_lower, r2_upper),
        'rmse_scores': rmse_scores,
        'mae_scores': mae_scores,
        'r2_scores': r2_scores
    }
    
    return bootstrap_results

# Realizar bootstrapping para el mejor modelo basado en RMSE
print("\n🔄 Ejecutando evaluación por bootstrapping para el mejor modelo...")

# Encontrar el mejor modelo basado en RMSE
best_model_name = min(resultados_base.items(), key=lambda x: x[1][0])[0]
print(f"Mejor modelo según RMSE: {best_model_name}")

# Conseguir instancia del mejor modelo
if 'XGBoost_Optuna' in best_model_name:
    best_model = mejor_xgb
elif 'LightGBM_Optuna' in best_model_name:
    best_model = mejor_lgbm
elif 'RandomForest_Optuna' in best_model_name:
    best_model = mejor_rf
elif 'CNN' in best_model_name:
    print("No se puede aplicar bootstrapping directamente al modelo CNN, solo a modelos tabulares.")
    best_model = None
elif 'ConvLSTM' in best_model_name:
    print("No se puede aplicar bootstrapping directamente al modelo ConvLSTM, solo a modelos tabulares.")
    best_model = None
else:
    # Obtener el modelo original de modelos_base
    model_class = best_model_name.split('_')[0]
    if model_class == 'RandomForest':
        best_model = modelos_base['RandomForest']
    elif model_class == 'XGBoost':
        best_model = modelos_base['XGBoost']
    elif model_class == 'LightGBM':
        best_model = modelos_base['LightGBM']
        
# Aplicar bootstrapping al mejor modelo si es tabular
if best_model is not None:
    bootstrap_results = bootstrap_evaluate_model(best_model, X_test_scaled, y_test.values, n_iterations=100)
    
    # Visualizar distribución de RMSE por bootstrapping
    plt.figure(figsize=(12, 6))
    
    # Histograma de RMSE
    plt.subplot(1, 3, 1)
    plt.hist(bootstrap_results['rmse_scores'], bins=20, alpha=0.7, color='skyblue')
    plt.axvline(bootstrap_results['rmse_median'], color='red', linestyle='--', label=f"Mediana: {bootstrap_results['rmse_median']:.3f}")
    plt.axvline(bootstrap_results['rmse_ci'][0], color='green', linestyle=':', label=f"IC 95%: [{bootstrap_results['rmse_ci'][0]:.3f}, {bootstrap_results['rmse_ci'][1]:.3f}]")
    plt.axvline(bootstrap_results['rmse_ci'][1], color='green', linestyle=':')
    plt.title(f'Distribución de RMSE - {best_model_name}')
    plt.xlabel('RMSE')
    plt.ylabel('Frecuencia')
    plt.legend()
    
    # Histograma de MAE
    plt.subplot(1, 3, 2)
    plt.hist(bootstrap_results['mae_scores'], bins=20, alpha=0.7, color='lightgreen')
    plt.axvline(bootstrap_results['mae_median'], color='red', linestyle='--', label=f"Mediana: {bootstrap_results['mae_median']:.3f}")
    plt.axvline(bootstrap_results['mae_ci'][0], color='green', linestyle=':', label=f"IC 95%: [{bootstrap_results['mae_ci'][0]:.3f}, {bootstrap_results['mae_ci'][1]:.3f}]")
    plt.axvline(bootstrap_results['mae_ci'][1], color='green', linestyle=':')
    plt.title(f'Distribución de MAE - {best_model_name}')
    plt.xlabel('MAE')
    plt.legend()
    
    # Histograma de R²
    plt.subplot(1, 3, 3)
    plt.hist(bootstrap_results['r2_scores'], bins=20, alpha=0.7, color='salmon')
    plt.axvline(bootstrap_results['r2_median'], color='red', linestyle='--', label=f"Mediana: {bootstrap_results['r2_median']:.3f}")
    plt.axvline(bootstrap_results['r2_ci'][0], color='green', linestyle=':', label=f"IC 95%: [{bootstrap_results['r2_ci'][0]:.3f}, {bootstrap_results['r2_ci'][1]:.3f}]")
    plt.axvline(bootstrap_results['r2_ci'][1], color='green', linestyle=':')
    plt.title(f'Distribución de R² - {best_model_name}')
    plt.xlabel('R²')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(model_output_dir / 'bootstrap_distributions.png')
    plt.show()